In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()

csid = 12736

True

In [8]:
df_metadata = f'''
SELECT
   date_trunc('seconds',ts.start_time)::timestamp without time zone as start_time,
   ca.time_zone,
   vcs.collection_type,
   vcs.collection_set,
   ts.collection_set_id,
   ts.test_type_id,
   c.friendly_name AS carrier,
   ts.net_types,
   ts.call_network_type, -- Note: Substitute for `co_call_network_type`
   ts.flag_access_success,
   ts.flag_task_success,
   ts.dsd_effective_download_test_speed,
   ts.dsd_throughput_max,
   ts.dsu_effective_upload_test_speed,
   ts.percentage_access_success,
   ts.percentage_task_success,
   md2.fn_get_best_network_type(ts.test_type_id, 
                                          ts.net_types, 
                                          tea.network_types, 
                                          tea.call_network_type, 
                                          tea.nr_status_filtered, 
                                          tea.nr_bearer_status_filtered,
                                          tea.nr_bearer_allocation_status_filtered,
                                          ',') AS best_network_type,
   md2.fn_get_network_types(ts.net_types, ','::character varying) AS net_types_desc,
   kt.name AS kit_type,
   rsc.name AS report_set_collector,
   (ts.start_time AT TIME ZONE 'utc' AT TIME ZONE ca.time_zone) AS local_time
FROM prod_ms_partitions.test_summary_{csid} as ts
LEFT JOIN md2.collection_areas ca on (ts.collection_area_id = ca.collection_area_id)
LEFT JOIN md2.vi_collection_sets vcs on (ts.collection_set_id = vcs.collection_set_id)
LEFT JOIN md2.carriers c on (ts.carrier_id = c.carrier_id)
LEFT JOIN prod_rsr_partitions.test_event_aggr_{csid} tea on (ts.collection_set_id = tea.collection_set_id and ts.test_event_id = tea.test_event_id)
LEFT JOIN md2.kit_types kt on (kt.kit_type_id = ts.kit_type_id)
LEFT JOIN md2.report_set_collectors rsc on (rsc.report_set_collector_id = ts.report_set_collector_id)
WHERE ts.test_type_id IN (19, 20, 23) 
AND ts.collection_set_period_id IS NOT NULL 
AND ts.blacklisted = FALSE 
AND ts.flag_valid = TRUE 
AND ts.carrier_id <> 478
'''

df_test_summary_metadata_csid  = pd.read_sql_query(df_metadata, con=os.getenv('RSR_SVC_CONN'))

df_test_summary_metadata_csid.head()

,start_time,time_zone,collection_type,collection_set,collection_set_id,test_type_id,carrier,net_types,call_network_type,flag_access_success,...,dsd_effective_download_test_speed,dsd_throughput_max,dsu_effective_upload_test_speed,percentage_access_success,percentage_task_success,best_network_type,net_types_desc,kit_type,report_set_collector,local_time
0,2025-04-08 18:10:05,America/Chicago,RSR US National Route,Minnesota-US_2025-1H,12736,19,T-Mobile,4194304.0,None,None,...,NaN,NaN,47116.743111,1.0,1.0,NR SA,NR,Car,Galaxy S24,2025-04-08 13:10:05.004
1,2025-04-11 01:18:40,America/Chicago,RSR US National Route,Minnesota-US_2025-1H,12736,19,T-Mobile,4194304.0,None,None,...,NaN,NaN,80936.960000,1.0,1.0,NR SA,NR,Scout,Galaxy S24,2025-04-10 20:18:40.004
2,2025-04-11 01:56:35,America/Chicago,RSR US National Route,Minnesota-US_2025-1H,12736,23,T-Mobile,NaN,Unknown,False,...,NaN,NaN,NaN,NaN,NaN,Unknown,None,Car,Galaxy S24,2025-04-10 20:56:35.002
3,2025-04-08 17:47:10,America/Chicago,RSR US National Route,Minnesota-US_2025-1H,12736,20,Verizon,262144.0,None,None,...,64720.085183,99084.992,NaN,1.0,1.0,"NR NSA, LTE",LTE,Car,Galaxy S24,2025-04-08 12:47:10.003
4,2025-04-10 19:47:10,America/Chicago,RSR US National Route,Minnesota-US_2025-1H,12736,20,T-Mobile,4194304.0,None,None,...,306661.769912,456595.008,NaN,1.0,1.0,NR SA,NR,Car,Galaxy S24,2025-04-10 14:47:10.004
